In [8]:
from tweepy.streaming import StreamListener

from tweepy import OAuthHandler
from tweepy import API

from tweepy import Stream

import time
import json
import sys

import pandas as pd

# 1. Loading Data from Twitter

In [2]:
# Object, whicht tells how to handle incoming data; 
# It opens a new timestamped file in which to store tweets and takes an optional API argument;
# Required by Tweepy;

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [3]:
consumer_key = 'PWcjI6UyunGYmJJcabcjs8dll'
consumer_secret = 'oHYB2utS00hxIA0Ma8TBRW8ZC6fNIiDwowGfs9f80glNsXPWxH'
access_token = '1035271753122099205-TSAidHafn9E0kFsUCr4PTMSrjJebtg'
access_token_secret = 'cJfYsx03qvmNMxErX5LENegeXndCWKqZQED5mKT6tk1Xq'

In [4]:
# Authentication protocol, Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)
# Setting the access token and access token secret, Access key authentication
auth.set_access_token(access_token, access_token_secret)
# Pass the auth object to the tweepy API object
api = API (auth)

In [ ]:
# Set up words to track
keywords_to_track = ['#drosten', '#streeck']

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track)

# 2. Converting Twitter Data into Json Object

In [9]:
# Load JSON file into a JSON object
vir_json = open('twitter_data2.json', 'r').read()

# 3. Converting Json data into Python (Pandas Dataframe)

In [10]:
def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
        tweet_obj['user-location'] = tweet_obj['user']['location']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

In [11]:
# Flatten the tweets and store in `tweets`
tweets = flatten_tweets(vir_json)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [12]:
# Create a DataFrame from `tweets`
ds_tweets = pd.DataFrame(vir_json)

ValueError: DataFrame constructor not properly called!

In [ ]:
# Print out the first 5 tweets from this dataset
print(ds_tweets['text'].values[0:5])